In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf  #Use tensorflow.keras.preprocessing.text
from tensorflow.keras.preprocessing.text import one_hot #Use tensorflow.keras.preprocessing.text
from tensorflow.keras.preprocessing.sequence import pad_sequences #Use tensorflow.keras.preprocessing.sequence
from tensorflow.keras.models import Sequential #Use tensorflow.keras.models
from tensorflow.keras.callbacks import Callback #Use tensorflow.keras.callbacks
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #Use tensorflow.keras.layers
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df.iloc[:1000,:]

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,1
996,I hated it. I hate self-aware pretentious inan...,0
997,I usually try to be professional and construct...,0
998,If you like me is going to see this in a film ...,0


In [ ]:
def simple_hash(word,vocab_size):

  return ord(word[0])%vocab_size

word='hello'
vocab_size=100
hash_value=simple_hash(word,vocab_size)
print(hash_value)

4


In [ ]:
def custom_hash(word,vocab_size):
  hash_value=0
  prime=31


  for i ,char in enumerate(word):
    hash_value+=(ord(char)*prime**i)
  return abs (hash_value)%vocab_size +1

words=["I","LOVE","PROGRAMMING"]
vocab_size=100
hash_value=[custom_hash(word,vocab_size) for word in words]


print(f'original words:{words}')
print(f'hash values:{hash_value}')



original words:['I', 'LOVE', 'PROGRAMMING']
hash values:[74, 51, 26]


In [ ]:

x=np.array(df['review'])
y=np.array(df['sentiment'])

x_filtered=[]

for review in x:
  review = review.lower()
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i in review:
    if i in punc:
      review = review.replace(i, " ")
  x_filtered.append(review)

x_filtered[0]



'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me  br    br   the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word  br    br   it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away  br    br   i would say the main appeal of the show is due to the fa

In [ ]:
from sklearn.preprocessing import OneHotEncoder
vocabulary_size=5000
onehot_encoded = [one_hot(review,vocabulary_size)for review in x_filtered]
onehot_encoded
print(onehot_encoded)

Buffered data was truncated after reaching the output size limit.

In [ ]:
onehot_encoded_len=[len(review) for review in onehot_encoded]
max_len=max(onehot_encoded_len)
max_len

2525

# Padding

In [ ]:
max_lenght=500
x_padded=pad_sequences(onehot_encoded,maxlen=max_lenght,padding='post')
x_padded

array([[1249, 4557,  158, ...,    0,    0,    0],
       [3383, 3751, 1021, ...,    0,    0,    0],
       [1770, 3321, 1509, ...,    0,    0,    0],
       ...,
       [1770, 1900, 3383, ...,    0,    0,    0],
       [1770,  731, 2379, ...,    0,    0,    0],
       [4800, 1249, 3016, ...,    0,    0,    0]], dtype=int32)

# split data

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_padded,y,test_size=0.2,random_state=42)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=2)

Epoch 1/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 24ms/step - accuracy: 0.5033 - loss: 0.6931 - val_accuracy: 0.5058 - val_loss: 0.6880
Epoch 2/2
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.5208 - loss: 0.6823 - val_accuracy: 0.5175 - val_loss: 0.6851


# building and training the model

In [ ]:
model=Sequential()
embedded_vector_size=35
model.add(Embedding(vocabulary_size,embedded_vector_size,input_length=max_lenght))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


## Evaluate Model Accuracy on Test Data

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 50.25%


## Predict Sentiment on New Reviews

In [ ]:
def preprocess_review(review, vocabulary_size, max_lenght):
    """Preprocesses a single review."""
    review = review.lower()
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for i in review:
        if i in punc:
            review = review.replace(i, " ")

    onehot_encoded_review = one_hot(review, vocabulary_size)
    padded_review = pad_sequences([onehot_encoded_review], maxlen=max_lenght, padding='post')[0]
    return padded_review

In [ ]:
# Example of new reviews
new_reviews = [
    "This movie was fantastic! I loved every minute of it.",
    "The acting was terrible and the plot was boring.",
    "It was an okay movie, nothing special.",
    "A truly brilliant film that captured the essence of the book."
]

# Preprocess the new reviews
preprocessed_new_reviews = np.array([preprocess_review(review, vocabulary_size, max_lenght) for review in new_reviews])

# Make predictions
predictions = model.predict(preprocessed_new_reviews)

# Convert predictions to sentiment labels
predicted_sentiments = ["positive" if pred > 0.5 else "negative" for pred in predictions]

# Display the new reviews and their predicted sentiments
for review, sentiment in zip(new_reviews, predicted_sentiments):
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step
Review: This movie was fantastic! I loved every minute of it.
Predicted Sentiment: negative

Review: The acting was terrible and the plot was boring.
Predicted Sentiment: negative

Review: It was an okay movie, nothing special.
Predicted Sentiment: negative

Review: A truly brilliant film that captured the essence of the book.
Predicted Sentiment: negative



## Modify model architecture

### Subtask:
Introduce a Bidirectional LSTM layer for better context understanding.


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download punkt_tab

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

x_cleaned = []
for review in x_filtered:
    tokens = word_tokenize(review)
    cleaned_review = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    x_cleaned.append(" ".join(cleaned_review))

x_filtered = x_cleaned

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
embedded_vector_size = 128
model.add(Embedding(vocabulary_size, embedded_vector_size, input_length=max_lenght))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


## Retrain the model

### Subtask:
Train the modified model with the improved data and architecture.


In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=7)

Epoch 1/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 54s 41ms/step - accuracy: 0.5925 - loss: 0.6726 - val_accuracy: 0.7636 - val_loss: 0.5168
Epoch 2/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.7590 - loss: 0.5121 - val_accuracy: 0.8306 - val_loss: 0.4044
Epoch 3/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.8442 - loss: 0.3751 - val_accuracy: 0.8476 - val_loss: 0.3569
Epoch 4/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.8812 - loss: 0.2956 - val_accuracy: 0.8729 - val_loss: 0.3063
Epoch 5/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.9096 - loss: 0.2364 - val_accuracy: 0.8745 - val_loss: 0.3019
Epoch 6/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.9202 - loss: 0.2128 - val_accuracy: 0.8820 - val_loss: 0.2916
Epoch 7/7
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.9456 - loss: 0.1586 - val_accuracy: 0.8805 - val_loss: 0.3186


In [ ]:
display(df.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


None

In [ ]:
display(df.duplicated().sum())

np.int64(418)

In [ ]:
df.drop_duplicates(inplace=True)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 49582 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     49582 non-null  object
 1   sentiment  49582 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


None

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 88.05%
